In [0]:
import os
os.chdir("./drive/My Drive/179skill")

In [0]:
import numpy as np
import pystan
import matplotlib.pyplot as plt
import random
import time
%matplotlib inline

In [0]:
def logit(z): return 1./(1.+np.exp(-z))

In [0]:
def generate_players(num_of_players, skill_cap):
    ## returns a list of player's skill
    return [round(random.uniform(0,skill_cap),2) for i in range(num_of_players)]


def generate_games(players, num_of_games, scale=0.3, style='pystan'):

    if style == 'pystan':
        
        player1=[]
        player2=[]
        outcome=[]
        for i in range(num_of_games):
            p1,p2 = random.sample(range(len(players)),2)
            win_rate=logit(scale*(players[p1]-players[p2]) )

            ##pystan player id is ONE based
            player1.append(p1+1)
            player2.append(p2+1)
            
            outcome.append(*random.choices([1,0],weights=[win_rate,1-win_rate]))

        return player1,player2,outcome
    
    elif style == 'pygm':
        games=[]
        for i in range(num_of_games):
            p1,p2 = random.sample(range(len(players)),2)
            win_rate=logit(scale*(players[p1]-players[p2]) )
            
            games.append((p1,p2,*random.choices([1,-1],weights=[win_rate,1-win_rate])))
        return games
    
    assert False 


In [0]:
def prediction_accuracy(skill_data, samples, true_players, num_valid_game=1000, scale=0.3):
    
    valid_games = generate_games(true_players, num_valid_game, scale, style='pystan')

    acc=0
    for g in range(num_valid_game):
        i,j,result=valid_games[0][g],valid_games[1][g],valid_games[2][g]

        prediction = logit( skill_data['scale']*(samples['skill'][:,i-1]-samples['skill'][:,j-1]) ).mean()

        acc+= int( (prediction >= 0.5 and result==1) or (prediction<0.5 and result==0) )
    return acc/num_valid_game

In [0]:
skill_model = """
data {
  int<lower=1> N;             # Total number of players
  int<lower=1> E;             # number of games
  real<lower=0> scale;        # scale value for probability computation
  int<lower=0,upper=1> win[E]; # PA wins vs PB
  int PA[E];                  # player info between each game
  int PB[E];                  # 
}
parameters {
  vector [N] skill;           # skill values for each player
}

model{
  for (i in 1:N){ skill[i]~normal(0,3); }
  for (i in 1:E){
    win[i] ~ bernoulli_logit( (scale)*(skill[PA[i]]-skill[PB[i]]) );
  }   # win probability is a logit function of skill difference
}
"""

In [0]:
import pickle
try:     # load it if already compiled
    sm = pickle.load(open('skill_model.pkl', 'rb'))
except:  # ow, compile and save compiled model
    sm = pystan.StanModel(model_code = skill_model)
    with open('skill_model.pkl', 'wb') as f: pickle.dump(sm, f)

In [10]:

scale=0.5
num_of_players=20
result={}
for num_of_games in [10, 20, 50,100,500,700,1000,3000,5000]:
    starting_num_of_games=time.time()
    trials_result=[]
    print("starting num_of_games =",num_of_games)
    for trials in range(10):
        true_players = generate_players(num_of_players,10)
        train_games = generate_games(true_players, num_of_games, scale, style='pystan')
        # print("true skills:",true_players)
        skill_data = {
            'N': num_of_players,
            'E': num_of_games,
            'scale': scale,
            'win':train_games[2],
            'PA': train_games[0],
            'PB': train_games[1]
        }
        starting=time.time()
        fit = sm.sampling(data=skill_data, iter=10000, chains=num_of_players)
        print("spent:",round(time.time()-starting,3))
        samples = fit.extract()
        # print("predicted skills:",samples['skill'].mean(0))
        acc=prediction_accuracy(skill_data, samples, true_players, num_valid_game=10000, scale=scale)
        print("acc:",acc,"\n")
        trials_result.append(acc)
        
    result[num_of_games]=trials_result
    print("Complete",num_of_games,"| Spent:",round(time.time()-starting_num_of_games,2))
print(result)

starting num_of_games = 10
spent: 22.545
acc: 0.5574 

spent: 21.505
acc: 0.5831 

spent: 15.253
acc: 0.5816 

spent: 9.245
acc: 0.571 

spent: 8.994
acc: 0.566 

spent: 9.203
acc: 0.6404 

spent: 9.407
acc: 0.554 

spent: 9.126
acc: 0.6296 

spent: 9.187
acc: 0.6048 

spent: 13.75
acc: 0.5975 

Complete 10 | Spent: 431.77
starting num_of_games = 20
spent: 10.968
acc: 0.6049 

spent: 11.049
acc: 0.6799 

spent: 10.795
acc: 0.647 

spent: 11.001
acc: 0.6028 

spent: 10.814
acc: 0.6478 

spent: 10.83
acc: 0.6714 

spent: 10.829
acc: 0.6051 

spent: 11.014
acc: 0.6568 

spent: 10.953
acc: 0.5898 

spent: 10.927
acc: 0.6443 

Complete 20 | Spent: 345.82
starting num_of_games = 50
spent: 16.338
acc: 0.672 

spent: 16.428
acc: 0.7144 

spent: 16.417
acc: 0.6796 

spent: 16.587
acc: 0.7275 

spent: 16.662
acc: 0.659 

spent: 16.734
acc: 0.6609 

spent: 16.388
acc: 0.6664 

spent: 16.537
acc: 0.6749 

spent: 16.422
acc: 0.7401 

spent: 16.5
acc: 0.7212 

Complete 50 | Spent: 402.76
starting nu

20 players, generating [10, 20, 50,100,500,700,1000,3000,5000] games

In [11]:
print(result)

{10: [0.5574, 0.5831, 0.5816, 0.571, 0.566, 0.6404, 0.554, 0.6296, 0.6048, 0.5975], 20: [0.6049, 0.6799, 0.647, 0.6028, 0.6478, 0.6714, 0.6051, 0.6568, 0.5898, 0.6443], 50: [0.672, 0.7144, 0.6796, 0.7275, 0.659, 0.6609, 0.6664, 0.6749, 0.7401, 0.7212], 100: [0.7212, 0.7055, 0.7743, 0.744, 0.7466, 0.7331, 0.7452, 0.7447, 0.7527, 0.7331], 500: [0.7856, 0.7695, 0.7802, 0.7854, 0.8056, 0.8078, 0.7683, 0.7625, 0.7846, 0.7978], 700: [0.7869, 0.7858, 0.737, 0.7817, 0.7873, 0.7986, 0.8018, 0.8016, 0.7894, 0.7745], 1000: [0.783, 0.7864, 0.7585, 0.7494, 0.8155, 0.7569, 0.7974, 0.7736, 0.7727, 0.7704], 3000: [0.7802, 0.7926, 0.7312, 0.8129, 0.798, 0.7802, 0.8134, 0.7489, 0.7709, 0.79], 5000: [0.8068, 0.7759, 0.7664, 0.8047, 0.7491, 0.8092, 0.777, 0.8183, 0.7394, 0.776]}


In [12]:
def avg(x):
    return sum(x)/len(x)
for i in result:
    print(i,avg(result[i]))

10 0.58854
20 0.6349800000000001
50 0.6916
100 0.7400399999999999
500 0.78473
700 0.7844599999999999
1000 0.77638
3000 0.78183
5000 0.7822799999999999
